In [1]:
import os
import functools

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tests.metaworld.envs.mujoco.sawyer_xyz.utils import trajectory_summary
from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise
import gym
gym.logger.set_level(40)
sns.set()
%load_ext autoreload
%autoreload 2

mujoco here!
mujoco now here!
mujoco here!
mujoco now here!


/home/ykai/anaconda3/envs/metaGarage/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
def sample_trajectories_from(env, policy, act_noise_pct, iters=100):
    sampled_rewards = []
    sampled_returns = []
    sampled_first_successes = []

    for _ in range(iters):
        s = trajectory_summary(env, policy, act_noise_pct, end_on_success=False)
        sampled_rewards.append(s[1])
        sampled_returns.append(s[2])
        sampled_first_successes.append(s[3])

    sampled_rewards = np.vstack(sampled_rewards)
    sampled_returns = np.vstack(sampled_returns)
    sampled_first_successes = np.array(sampled_first_successes)

    return sampled_rewards, sampled_returns, sampled_first_successes


def plot(rewards, returns, first_successes, tag):
    first_success = min(int(first_successes.mean()), rewards.shape[1])
    first_success_rew = rewards.mean(axis=0)[first_success]
    first_success_ret = returns.mean(axis=0)[first_success]
    
    fig, ax = plt.subplots(1, 2, figsize=(6.75, 4))

    reward_df = pd.DataFrame(rewards).melt()
    ax[0] = sns.lineplot(x='variable', y='value', data=reward_df, ax=ax[0], ci=95, lw=.5)
    ax[0].set_xlabel('Time Steps')
    ax[0].set_ylabel('Reward')
    ax[0].set_title('Rewards')
    ax[0].vlines(first_success, ymin=0, ymax=first_success_rew, linestyle='--', color='green')
    ax[0].hlines(first_success_rew, xmin=0, xmax=first_success, linestyle='--', color='green')
#     ax[0].set_yscale('symlog')

    return_df = pd.DataFrame(returns).melt()
    ax[1] = sns.lineplot(x='variable', y='value', data=return_df, ax=ax[1], ci=95, lw=.5)
    ax[1].set_xlabel('Time Steps')
    ax[1].set_ylabel('Return')
    ax[1].set_title('Returns')
    ax[1].vlines(first_success, ymin=0, ymax=first_success_ret, linestyle='--', color='green')
    ax[1].hlines(first_success_ret, xmin=0, xmax=first_success, linestyle='--', color='green')
#     ax[1].set_yscale('symlog')

    plt.subplots_adjust(top=.85)
    fig.suptitle(f'{tag} (n={rewards.shape[0]})')
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    plt.show()
    if not os.path.exists('figures'):
        os.mkdir('figures')
    fig.savefig(f'figures/{tag}_rewards_returns.jpg')

In [3]:
%reload_ext autoreload
config = [
    # env name, action noise, path length
    ['pick-place-v2', np.zeros(4), 200],
]

for env, noise, path_length in config:
    tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)

    policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
    env = ALL_ENVS[env]()
    env.max_path_length = path_length
    env._partially_observable = False
    env._freeze_rand_vec = False
    env._set_task_called = True

    sampled_rewards, sampled_returns, sampled_first_successes = sample_trajectories_from(env, policy, noise)
    plot(sampled_rewards, sampled_returns, sampled_first_successes, tag)

AssertionError: Obs Out of Bounds
	low: [-0.525   0.348  -0.0525 -0.1     0.8     0.05  ], 
	obs: [0.00615235 0.6001898  0.19430117 0.06398421 0.87266495 0.10471533], 
	high: [0.525 1.025 0.7   0.1   0.9   0.3  ]

In [4]:
i = 12
first_success = sampled_first_successes[i]
first_success_reward = sampled_rewards[i][first_success]
fig, ax = plt.subplots(1, 1, figsize=(6.75, 4))
ax.plot(np.arange(len(sampled_rewards[0])), sampled_rewards[i])

ax.vlines(first_success, ymin=0, ymax=first_success_reward, linestyle='--', color='green')
ax.hlines(first_success_reward, xmin=0, xmax=first_success, linestyle='--', color='green')

NameError: name 'sampled_first_successes' is not defined